# FB Prophet Model - Wage Rate ($)

Training (1997-2020), Training (2021-2023)

# Packages & Data Loading

## Packages

In [1]:
# Import packages
import pandas as pd
import numpy as np
from prophet import Prophet

import gdown
import requests
from io import StringIO

from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go

## Data Loading

In [2]:
# Google Drive file
file_id = '117pk3NKZPfYmHQyP452HeMV7_8Jq6_W0'

# Create the download URL
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Use requests to get file content
response = requests.get(download_url)
response.raise_for_status()  

# Load the CSV directly into df
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)

# Display data
print('Shape of the dataset:', df.shape)
print(df.dtypes)

Shape of the dataset: (349920, 14)
ref_date                   object
geo                        object
labour_force               object
industry                   object
sex                        object
age_group                  object
value_unemployment        float64
wages                      object
value_wage                float64
year                        int64
month                       int64
ref_date_numeric          float64
value_unemployment_log    float64
value_wage_log            float64
dtype: object


In [3]:
# Create a copy with selected columns
df_var = df[['ref_date', 'value_wage']].copy()

print(df_var.head())

     ref_date  value_wage
0  1997-01-01        7.00
1  1997-01-01        8.00
2  1997-01-01        7.50
3  1997-01-01        7.00
4  1997-01-01        9.49


In [4]:
# Group df_var by 'ref_date' and calculate the median 'value_unemployment'
df_fb = df_var.groupby('ref_date').agg(
    median_unemployment=('value_wage', 'median')  # Calculate the median 
).reset_index()

print(df_fb)

       ref_date  median_unemployment
0    1997-01-01            12.375000
1    1997-02-01            12.695000
2    1997-03-01            12.511667
3    1997-04-01            12.585000
4    1997-05-01            12.000000
..          ...                  ...
319  2023-08-01            26.037639
320  2023-09-01            27.000000
321  2023-10-01            26.670000
322  2023-11-01            27.000000
323  2023-12-01            27.400000

[324 rows x 2 columns]


In [5]:
# Display shape and data types 
print('Shape of the dataset:\n', df_var.shape)
print(df_fb.dtypes)

# Convert 'ref_date' to datetime
df_fb['ref_date'] = pd.to_datetime(df_fb['ref_date'])

# Set 'ref_date' as the index 
df_fb = df_fb.set_index('ref_date')

# Verify the index is set and DatetimeIndex
print('\n\nIndex after setting ref_date:\n', df_fb.head().index)
print('\n\nData types after setting index:\n', df_fb.dtypes)
print('\n\nDataset:\n', df_fb.head(2))

Shape of the dataset:
 (349920, 2)
ref_date                object
median_unemployment    float64
dtype: object


Index after setting ref_date:
 DatetimeIndex(['1997-01-01', '1997-02-01', '1997-03-01', '1997-04-01',
               '1997-05-01'],
              dtype='datetime64[ns]', name='ref_date', freq=None)


Data types after setting index:
 median_unemployment    float64
dtype: object


Dataset:
             median_unemployment
ref_date                       
1997-01-01               12.375
1997-02-01               12.695


# FB Prophet Model

## Assign Training & Testing Sets

In [6]:
# Split into training and testing sets
train = df_fb['1997-01-01':'2020-12-01']
test = df_fb['2021-01-01':'2023-12-01']

print(train.head())
print(train.tail())

            median_unemployment
ref_date                       
1997-01-01            12.375000
1997-02-01            12.695000
1997-03-01            12.511667
1997-04-01            12.585000
1997-05-01            12.000000
            median_unemployment
ref_date                       
2020-08-01            23.463571
2020-09-01            24.927500
2020-10-01            24.897857
2020-11-01            24.281875
2020-12-01            24.040000


In [7]:
# Prepare data for Prophet
train = train.reset_index()  # Reset index to make 'ref_date' a column
train.rename(columns={'ref_date': 'ds', 'median_unemployment': 'y'}, inplace=True)

# Prepare data for Prophet
test = test.reset_index()  # Reset index to make 'ref_date' a column
test.rename(columns={'ref_date': 'ds', 'median_unemployment': 'y'}, inplace=True)

## Model Fitting

In [8]:
# Fit model
model = Prophet()
model.fit(train)

20:10:01 - cmdstanpy - INFO - Chain [1] start processing
20:10:02 - cmdstanpy - INFO - Chain [1] done processing


# Cross Validation

In [9]:
# Cross validation
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon = '365 days')
df_p = performance_metrics(df_cv)
df_p.head()

  0%|          | 0/43 [00:00<?, ?it/s]

20:10:02 - cmdstanpy - INFO - Chain [1] start processing
20:10:16 - cmdstanpy - INFO - Chain [1] done processing
20:10:16 - cmdstanpy - INFO - Chain [1] start processing
20:10:17 - cmdstanpy - INFO - Chain [1] done processing
20:10:17 - cmdstanpy - INFO - Chain [1] start processing
20:10:17 - cmdstanpy - INFO - Chain [1] done processing
20:10:17 - cmdstanpy - INFO - Chain [1] start processing
20:10:17 - cmdstanpy - INFO - Chain [1] done processing
20:10:18 - cmdstanpy - INFO - Chain [1] start processing
20:10:18 - cmdstanpy - INFO - Chain [1] done processing
20:10:18 - cmdstanpy - INFO - Chain [1] start processing
20:10:18 - cmdstanpy - INFO - Chain [1] done processing
20:10:19 - cmdstanpy - INFO - Chain [1] start processing
20:10:19 - cmdstanpy - INFO - Chain [1] done processing
20:10:19 - cmdstanpy - INFO - Chain [1] start processing
20:10:19 - cmdstanpy - INFO - Chain [1] done processing
20:10:20 - cmdstanpy - INFO - Chain [1] start processing
20:10:20 - cmdstanpy - INFO - Chain [1]

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,40 days,0.092307,0.303820,0.237168,0.013935,0.010896,0.013959,0.535948
1,41 days,0.091762,0.302922,0.236751,0.013895,0.010896,0.013918,0.529412
2,42 days,0.086218,0.293629,0.226030,0.013189,0.010088,0.013223,0.568627
3,44 days,0.085186,0.291866,0.223852,0.013157,0.010088,0.013191,0.558824
4,45 days,0.086308,0.293782,0.225878,0.013381,0.010834,0.013408,0.549020


# Model Predictions

In [10]:
# Predictions
future = model.make_future_dataframe(periods=36, freq='ME')
future.tail()

,ds
319,2023-07-31
320,2023-08-31
321,2023-09-30
322,2023-10-31
323,2023-11-30


In [11]:
# Prediction columns 
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
319,2023-07-31,24.797886,24.410634,25.172691
320,2023-08-31,25.831992,25.455347,26.161620
321,2023-09-30,26.216291,25.848812,26.583493
322,2023-10-31,26.468455,26.091309,26.851335
323,2023-11-30,26.581170,26.204426,26.982092


In [12]:
# Merge actuals and predictions
evaluation = test.copy()
evaluation['yhat'] = forecast['yhat']
evaluation['yhat_lower'] = forecast['yhat_lower']
evaluation['yhat_upper'] = forecast['yhat_upper']

# Calculate performance metrics
actuals = evaluation['y'].values
predictions = evaluation['yhat'].values

# Metrics calculation
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)

# Handle zero or near-zero actuals for MAPE
epsilon = 1e-10
valid_indices = actuals > epsilon
if valid_indices.any():
    mape = np.mean(np.abs((actuals[valid_indices] - predictions[valid_indices]) / (actuals[valid_indices] + epsilon))) * 100
else:
    mape = np.nan

# Print final model evaluation statistics
print(f"Final Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}")

Final Model Evaluation:
Mean Squared Error (MSE): 170.9103261056854
Root Mean Squared Error (RMSE): 13.07326761393973
Mean Absolute Error (MAE): 13.05885653276839
Mean Absolute Percentage Error (MAPE): 51.0848137106567


# Visualizations

In [13]:
# Generate plot
fig = plot_plotly(model, forecast)

# Add labels and title
fig.update_layout(
    title="Prophet Forecast",                 
    xaxis_title="Date",                       
    yaxis_title="Median Unemployment",        
    legend_title="Legend",                   
    font=dict(size=12),                       
)

# Show plot
fig.show()

In [14]:
# Generate plot
fig = plot_plotly(model, forecast)

# Add test data 
fig.add_trace(
    go.Scatter(
        x=test['ds'],  # x-axis (test data dates)
        y=test['y'],  # y-axis (actual values)
        mode='markers+lines',
        name='True Values (Test Data)',
        line=dict(color='red', dash='dot'),  
        marker=dict(color='red')
    )
)

# Adjust x-axis range (2021-2023)
fig.update_layout(
    xaxis=dict(
        range=['2021-01-01', '2023-12-31'],  # Set x-axis range
        title='Date',  # Add x-axis title
    ),
    yaxis=dict(
        title='Median Unemployment Rate',  # Add y-axis title
    ),
    title='Prophet Forecast with Test Data (2021-2023)',  # Add plot title
)

# Show plot
fig.show()

In [15]:
# Generate plot
fig = plot_components_plotly(model, forecast)

# Add title and labels
fig.update_layout(
    title="Prophet Forecast",     
    font=dict(size=12),                      
)       

# Show plot
fig.show()